# Problema del commesso viaggiatore - Tabu Search

## Import ed inizializzazioni

Import e parametri

In [80]:
import copy # Per eseguire una copia negli stati
MAX_CONT = 10 # Per la condizione di terminazione
TENURE = 3 # Tabu tenure

Inizializzazione dello stato e del tenure

In [81]:
state = ['A', 'B', 'C', 'D', 'E']

Inizializzazione delle connessione

In [82]:
connections = {}
connections['A'] = [['B', 4], ['C', 2], ['D',  1], ['E',  2]]
connections['B'] = [['A', 4], ['C', 1], ['D', 2], ['E', 3]]
connections['C'] = [['A', 2], ['B', 1], ['D', 3], ['E',  2]]
connections['D'] = [['A', 1], ['B', 2], ['C',  3], ['E',  2]]
connections['E'] = [['A', 2], ['B',  3], ['C',  2], ['D',  2]]

## Funzioni

Funzione di valutazione

In [83]:
def eval_state(state):
    len_path = 0 # lunghezza del percorso
    dim= len(state)
    for ind in range(dim):
        ls = connections[state[ind]] # lista di tutte le connessioni
        if ind < (dim-1): # dim - 1 perché prendi l'indice successivo
            # Ancora non ha visitato tutte le citta'
            succ_node = state[ind + 1]
        else:
            # Ha visitato tutte le citta'
            succ_node = state[0]
        # Cerca la distanza nel successore
        for (succ, dist) in ls:
            if succ == succ_node:
                # Nodo successore trovato
                len_path += dist # somma la distanza
                break
    return len_path

Funzione per la generazione del vicinato

In [84]:
def gen_succ(state):
    # Genera la lista ordinata di successori di uno stato
    ls = []
    dim = len(state)
    # Generazione dei successori
    for ind in range(0, dim-1):
        for inx in range(ind+1, dim):
            # Copia l'intera struttura dello stato (deepcopy)
            buffer = copy.deepcopy(state)
            # Scambia i due elementi del buffer
            temp = buffer[ind]
            buffer[ind] = buffer[inx]
            buffer[inx] = temp
            # Valuta i successori e aggiungilo nella lista
            eval_succ = eval_state(buffer)
            ls.append((buffer, eval_succ, (state[ind], state[inx])))
    ls.sort(key= lambda x : x[1]) # Ordina lista in base a eval_succ (x[1])
    return ls

## Algoritmo vero e proprio

Test di assenza mossa

In [85]:
def tabu_test(seq, tabu_list):
    a, b = seq[2]
    if ((a,b) in tabu_list or (b,a) in tabu_list):
        absent = False
    else:
        absent = True
    return absent

Algoritmo

In [ ]:
def tabu_search(state, tabu_tenure, connections):
    # Stato iniziale
    current = state
    eval_current = eval_state(current)
    # Inizializza la best
    best = current
    eval_best = eval_current
    # Inizializzazione della lista tabu e del vicinato
    tabu_list = {}
    neighbors = []
    # Contatore per la condizione di terminazione
    cont = 0
    # Esecuzione dell'algoritmo
    while cont < MAX_CONT:
        succ = gen_succ(current) # lista di successori
        # Seleziona i successori che non si trovano nella tabu
        neighbors = list(filter(lambda n : tabu_test(n, tabu_list), succ))
        # Elementi di neighbors:
        # I colonna: stato successivo
        # II colonna: valutazione successiva
        # III colonna: mossa successiva
        next = neighbors[0][0]
        eval_next = neighbors[0][1]
        # Controllo se il successore e' meglio del best
        delta = eval_best - eval_next
        if delta > 0:
            # Trovato successore migliore
            best = next
            eval_best = eval_next
        # Aggiorna il successore
        current = next
        eval_current = eval_next
        # Decrementa la tabu tenore e 
        # elimina gli elementi con tenure nulli
        for move in tabu_list:
            tabu_list[move] -= 1
        tabu_list = {k : tabu_list[k] for k in tabu_list if tabu_list[k] != 0}
        # Inserisci la prossima mossa nella tabu list
        next_move = neighbors[0][2]
        tabu_list[next_move] = tabu_tenure
        # Aggiorna il contatore
        cont += 1
    # Aggiungi lo stato iniziale
    best = best + [best[0]]
    return (best, eval_best)

## Main

In [87]:
tabu_search(state, TENURE, connections)

(['C', 'B', 'D', 'A', 'E', 'C'], 8)